In [25]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [26]:
#additional 
import nltk
import re
import string
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [27]:
train=pd.read_csv("../input/korea-university-cose362-2021-fall/train.csv")
train.head(10)

In [28]:
print (train.shape) #the train.csv has 3620 row and 3 cells (3620, 3)
print (train.columns) #Index(['id', 'label', 'mail'], dtype='object')

In [29]:
test=pd.read_csv("../input/korea-university-cose362-2021-fall/test.csv")
test.head()

In [30]:
print (test.shape) #the train.csv has 3620 row and 3 cells (1151, 2)
print (test.columns) #Index(['id', 'mail'], dtype='object')

In [31]:
train=train[['id','label','mail']]
train.head()

In [32]:
train.isnull().sum

In [33]:
stopwards=set(stopwords.words(['danish', 'dutch', 'english', 'finnish', 'french', 'german', 'hungarian', 'italian', 'norwegian', 'portuguese', 'russian', 'spanish', 'swedish', 'turkish']))
def filter_mail(mail):
    #change to lower case
    #delete punctuation, URL, special characte, stopwords
    #return clean textwords
    mail=mail.lower()
    mail=re.sub("@\w+(?:[-']\w+)*","" "",mail)
    mail=re.sub("[#]']"," ",mail)
    mail=re.sub("http\S+"," ",mail)
    mail=re.sub(r'^0-9a-zA-Z',' ',mail)
    mail=re.sub(r'\s+',' ',mail)
    nopunc=[char for char in mail if char not in string.punctuation]
    nopunc=''.join(nopunc)
    filtered_words=[word for word in nopunc.split() if word.lower() not in stopwords.words(['danish', 'dutch', 'english', 'finnish', 'french', 'german', 'hungarian', 'italian', 'norwegian', 'portuguese', 'russian', 'spanish', 'swedish', 'turkish'])]
    return filtered_words

In [34]:
train['mail'].head().apply(filter_mail)
test['mail'].head().apply(filter_mail)
train.head()

In [35]:
X=train['mail']
y=train['label']

In [36]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix

def classification(model,X, y):
    #split
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0, shuffle=True, stratify=y)
    #train
    pip_model = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', model)])
    pip_model.fit(x_train, y_train)
    
    print('Accuracy:', pip_model.score(x_test, y_test)*100)
    y_pred = pip_model.predict(x_test)
    print(classification_report(y_test, y_pred))
    print (confusion_matrix(y_test, y_pred))
    

In [37]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
classification(model, X, y)

In [38]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
classification(model, X, y)

In [39]:
from sklearn.svm import SVC
model = SVC(C=3)
classification(model, X, y)

In [40]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
classification(model, X, y)

In [41]:
from sklearn.neural_network import MLPClassifier
model=MLPClassifier()
classification(model, X, y)

In [42]:
from sklearn.linear_model import SGDClassifier
model=SGDClassifier()
classification(model, X, y)

In [43]:
from sklearn.linear_model import SGDClassifier
model=SGDClassifier(loss="hinge", penalty="l2", max_iter=100)
classification(model, X, y)

Find the max iteration

**Find the maxiteration when using SGDClassifierr**

In [44]:
train_data = train.drop('label', axis=1)
target = train['label']
train_data.head()

In [45]:
train_data.shape

In [ ]:
from sklearn.neural_network import MLPClassifier
model = MLPClassifier()
pip_model = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', model)])
pip_model.fit(X, y)
    
test_prediction=pip_model.predict(test['mail'])

print(test_prediction)
submission=pd.DataFrame({"id":test["id"],"label":test_prediction})
submission.to_csv('submission.csv',index=False)
submission =pd.read_csv('submission.csv')


**Visualization**

In [47]:
import matplotlib.pyplot as plt
print("Train.csv Data")
train['label'].value_counts().plot(kind='bar')
plt.xlabel('0:Non Spam 1:Spam')
plt.ylabel('Number of mails')
print(train.label.value_counts())

In [48]:
import matplotlib.pyplot as plt
import seaborn as sns
print("Test.csv Data")
submission['label'].value_counts().plot(kind='bar')
plt.xlabel('0:Non Spam 1:Spam')
plt.ylabel('Number of mails')
print(submission.label.value_counts())

In [51]:
from wordcloud import WordCloud
non_spam=test['mail'][submission['label']==0]
cloud = WordCloud(width=500,height=100,background_color='black',max_words=30).generate(' '.join(non_spam))
plt.figure(figsize=(10,8))
plt.imshow(cloud)
plt.axis('off')
plt.show()

In [50]:
from wordcloud import WordCloud
spam=test['mail'][submission['label']==1]
cloud=WordCloud(width=500,height=100,background_color='black',max_words=30).generate(' '.join(spam))
plt.figure(figsize=(10,8))
plt.imshow(cloud)
plt.axis('off')
plt.show()